# Visualization: Day 3 Interaction
## Learning Outcomes

By the end of this session, students will be able to:

1. **Explore trends dynamically** by zooming and panning across data.
2. **Highlight specific categories** to focus on targeted insights.
3. **Apply selection-based filtering** to reveal relevant details in linked visualizations.
4. **Create linked visualizations** using brush-controlled selection to connect and filter data across multiple charts.

### **Types of Interaction in This Tutorial**

1. **Zoom and Pan**: Adjust the view on both x and y axes, providing detailed access to specific data segments.
2. **Highlighting Categories**: Emphasize particular data categories within a chart, making it easy to focus on specific subsets.
3. **Selection-Based Filtering**: Use selections to filter data in linked visualizations, enabling dynamic exploration across charts.
4. **Brush-Controlled Linking**: Create a uni-directional link between two charts, where a selection in one chart filters data in another.


Now that you understand the basic grammar you can explore other examples on the web
 https://altair-viz.github.io/gallery/index.html
 
Here is the api https://altair-viz.github.io/user_guide/api.html
 
All of the customization we will do today can be found in the documentation
 
 

## Breaches Dataset and Environment Setup

In [ ]:
!pip install altair
import pandas as pd
import altair as alt

## Breaches Dataset
The original dataset is available here
https://docs.google.com/spreadsheets/d/1i0oIJJMRG-7t1GT-mr4smaTTU7988yXVz8nPlwaJ8Xk/edit?gid=2#gid=2

There are a series of visualizations that have been created off of this dataset
https://informationisbeautiful.net/visualizations/worlds-biggest-data-breaches-hacks/


In [ ]:
# Read in the data 
url_path = "https://raw.githubusercontent.com/kemiolamudzengi/viz_4_all/main/cleaned_breaches.csv"
breaches = pd.read_csv(url_path, parse_dates=True)
# Remove rows where 'sector' is in the specified list
breaches_filtered = breaches[~breaches['sector'].isin(['academic', 'legal', 'media', 'ngo', 'misc'])]


### Normalized Bar Chart
 - number of breaches on the x channel
 - method on the y channel
 - data sensistivity on the color channel

    Make it normalized 

In [ ]:
# Normalized Bar Chart
normalized_bar_chart = ...
normalized_bar_chart

## Interaction

### **Explanation of Selections in Altair**

Altair provides **selection types** to make charts interactive by allowing users to filter, highlight, or zoom into data. 
#### **1. Selection Types**

- **`selection_point`**:
   - Used for **selecting individual points** in a chart.
   - Ideal for cases where you want to select a single data point or category and have other charts or elements respond to this selection.
   - Example: Selecting a single bar in a bar chart to filter data in a linked chart.

- **`selection_interval`**:
   - Used for **selecting a range** of data, typically along an axis.
   - Allows users to select multiple points within a specified range, often by dragging over an area.
   - Ideal for **zooming and panning** when combined with `bind='scales'`.


#### **2. `add_params` Method**

The `.add_params()` method in Altair is used to **add a selection or parameter to a chart**. This enables interactivity in the visualization, allowing selections like `selection_point` or `selection_interval` to control elements in the chart or even in linked charts. By adding parameters, you can define how users interact with the data, making it possible to filter or highlight specific values.


#### **3. Binding in Selections**

**Binding** links a selection to the **scales** of the chart, allowing the selected range to control the zoom and pan. By setting `bind='scales'` in an interval selection, Altair allows users to adjust the viewing window on the chart dynamically. This enables users to:

- **Zoom in**: Select a narrower range to view finer details.
- **Pan**: Scroll horizontally or vertically to move across different sections of the data.


## Task 1: Zoom and Pan on Area Chart
### Basic area chart setup for breaches over time by method
 - year on the x channel
 - number of breaches on the y channel 
 - method on color channel 

In [ ]:
area_zoom_pan = ...

area_zoom_pan


### Adding Zoom and Pan Interaction
**What We Will Add**:
- Enable **zooming and panning** using Altair’s `selection_interval` with scale binding.

**How to Add It**:
1. Define a `selection_interval` to allow users to select an interval on the chart.
2. Use `bind='scales'` to link the selection to the chart’s axes, enabling zoom and pan functionality.


In [ ]:
# Enable zoom and pan interaction
area_zoom_pan = ...

area_zoom_pan

#two fingers to zoom
#three fingers to pan

In [ ]:
# add_params(alt.selection_interval(bind='scales')): 
# Adds an interval selection with bind='scales', allowing the user to zoom and pan on both the x and y axes.


## **Task 2: Highlighting Categories in a Line Chart**

In [ ]:
# Base line chart for breach trends by sensitivity level with customized legend labels
# Map data_sensitivity levels to descriptive labels
breaches_labeled = breaches_filtered.assign(
    sensitivity_label=breaches_filtered['data_sensitivity'].map({
        1: "1:Email address/Online info.",
        2: "2:SSN/Personal details",
        3: "3:Credit card info.",
        4: "4:Health/other personal records",
        5: "5:Full details"
    })
)

# Base line chart with customized legend labels
line_highlight = alt.Chart(breaches_labeled).mark_line(
    strokeWidth=3,
    point=alt.OverlayMarkDef(size=60)
).encode(
    x=alt.X('year(date):T').title('Year'),
    y=alt.Y('count():Q').title('Number of Breaches'),
    color=alt.Color('sensitivity_label:N').title('Data Sensitivity Level').scale(scheme='reds')
).properties(
    title="Breach Trends Over Time by Sensitivity Level",
    width=500,
    height=300
)

line_highlight

#### **Step 2: Adding the Highlighting Interaction**
**What We Will Add**:
- Highlight lines in the chart by `data_sensitivity` selection.
  
**How to Add It**:
1. Create a `selection_point` on `data_sensitivity` to allow highlighting.
2. Bind the selection to the legend, so users can select a category directly from the legend.
3. Adjust the `opacity` using a condition: fully opaque for selected lines, and partially transparent for others.


In [ ]:
# Define a point selection on sensitivity_label
sensitivity_highlight = alt.selection_point(fields=['sensitivity_label'], bind='legend')

# Add highlighting interaction
line_highlight = ...
line_highlight

## **Task 3: Selection-Based Filtering with Linked Charts**

In [ ]:
# Base Bar Chart
bar_chart_filter = alt.Chart(breaches_filtered).mark_bar().encode(
    x=alt.X('count()').title('Number of Breaches').axis(tickCount=4),
    y=alt.Y('method:N').title('Cause of Breach'),
    color=alt.Color('method:N').title('Cause of Breach')
).properties(
    title="Number of Breaches by Method",
    width=500,
    height=80
)

# Base Scatter Plot
scatter_filtered = alt.Chart(breaches_filtered).mark_circle(opacity=0.7).encode(
    x=alt.X('records_lost:Q').title('Records Lost').axis(tickCount=4),
    y=alt.Y('sector:N').title('Sector'),
    color=alt.Color('method:N').title('Cause of Breach'),
    size=alt.Size('records_lost:Q').title('Records Lost').scale(domain=[1e6, 1e9], range=[10, 1000]),
    tooltip=['organisation', 'date', 'records_lost', 'method', 'data_sensitivity']
).properties(
    title="Characteristics of Breaches by Sector and Records Lost",
    width=500,
    height=250
)

bar_chart_filter & scatter_filtered


### **Adding Selection-Based Filtering**

**What We Will Add**:
- Create a linked interaction between a bar chart and a scatter plot, using selection-based filtering.

**How to Add It**:
1. Define a `selection_point` on `method` for selecting a breach method.
2. Highlight the selected bar in the bar chart.
3. Filter the scatter plot to show only the data associated with the selected breach method.


In [ ]:
# Define a point selection for method
method_selection = alt.selection_point(fields=['method'], empty="all")

# Updated Bar Chart with Selection
bar_chart_filter = ...
# Scatter Plot with Filtering
scatter_filtered = ...

(bar_chart_filter & scatter_filtered)

## **Task 4: Brush-Controlled Linking with Year Circle and Pie Chart**
In this task, we’ll use a brush-controlled selection on a circle chart to filter data displayed in a linked pie chart. This setup will allow you to select a range of years on the circle chart, dynamically updating the pie chart to show records_lost by sector within the selected time range.

In [ ]:
# Base Tick Chart for selecting years
year_chart = alt.Chart(breaches_filtered).mark_circle(size=20).encode(
  #  x=alt.X('year(date):O').title('Year'),
     x=alt.X('year(date):T').title('Year 20XX').axis(format='%y'),  # Formats year to two digits
   
    size = alt.Size('sum(records_lost):Q').scale(domain=[1e4, 3e9], range=[10, 500]),
  ).properties(
    title="Select Year Range to Filter Pie Chart",
    width=450,
    height=30
)

# Base Pie Chart for records lost by sector
pie_chart = alt.Chart(breaches_filtered).mark_arc().encode(
    theta=alt.Theta('count():Q').title("Records Lost"),
    color=alt.Color('sector:N').title("Sector").scale(scheme='set3'),
    tooltip=['sector', 'sum(records_lost):Q','count():Q']
).properties(
    title="Count of Incidences by Sector (Filtered by Year Selection)",
    width=250,
    height=250
)

# Display charts side-by-side
year_chart & pie_chart

### Adding Brush-Controlled Linking

**What We Will Add**:
- Use a brush selection on a tick chart to filter data displayed in a pie chart.

**How to Add It**:
1. Define a `selection_interval` on the x-axis to create a brush for selecting a year range.
2. Highlight the selected years in the tick chart.
3. Filter the pie chart to display only records lost within the selected year range.


In [ ]:
# Define a brush selection for the x-axis (year)
brush = alt.selection_interval(encodings=['x'])

# Tick Chart with Brush Selection
year_chart =...

# Pie Chart Filtered by Brush Selection
pie_chart = ...

# Display the interactive linked charts
year_chart & pie_chart


## **Combining the Dashboard**
Finally, we’ll combine all interactive charts into a single dashboard view for ease of exploration.


In [ ]:
# Combine all interactive charts into a dashboard
(area_zoom_pan & line_highlight) | (bar_chart_filter & scatter_filtered) | (year_chart & pie_chart)
